# [LVV-T2221] - MTMount Interface to SAL
The objective of this test case is to verify 
the MTMount communication interface with SAL by testing the commands, events and telemetry defined by the latest version of the XML
all topic parameters are filled with data
the data are reasonable.

This test case will exercise the functionality of the Telescope Mount and meet the following criteria:
* Only requires the most current version of SAL
* Requires the use of the DDS and the EFD

[LVV-T2221]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2221

## Setting Up Test Environment

In [ ]:
test_case = "LVV-T2221"
test_exec = "LVV-EXXXX"

In [ ]:
%load_ext autoreload
%autoreload 2

import asyncio
import logging
import os
import yaml

import astropy.units as u
import numpy as np
import pandas as pd

from astropy.time import Time
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

from lsst_efd_client import EfdClient
from lsst.ts import salobj
from lsst.ts.idl.enums import MTMount

from lsst.sitcom import vandv

In [ ]:
exec_info = vandv.ExecutionInfo()
print(exec_info)

In [ ]:
client = vandv.efd.create_efd_client()

The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [ ]:
os.environ["LSST_DDS_HISTORYSYNC"] = "30"

This sets up the logger for the test.

In [ ]:
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

Getting unique index for Script

In [ ]:
log.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

log.info(f'The generated index is {index}')

Starts a Script controller which allows putting custom messages into the EFD for later analysis.Probably do not need it.

In [ ]:
start_time = datetime.now()
test_message = "MTMount Interface Test"

script = salobj.Controller("Script", index=index)
await script.start_task
print(f"{test_case} {test_exec} time to start is {datetime.now() - start_time} [s]")

This is how you start the remote for the CSC.

In [ ]:
domain = salobj.Domain()
print(domain)

Create the remote to control the MTMount and start the service

In [ ]:
mount = salobj.Remote(name="MTMount", domain=domain)
print(mount)
await asyncio.sleep(10)
await mount.start_task

# State transitions

Check the summaryState. If, its not in the enable state, Transition the MTMount into enabledState, otherwise do nothing. 

In [ ]:
e = mount.evt_summaryState.get()
print("MTMount now in state:", e.summaryState) 
# Summary State 2 = ENABLED
# Summary State 1 = DISABLED
if e.summaryState not in [2]:
    print("  Transitioning to ENABLED state")
    await salobj.set_summary_state(mount, salobj.State.ENABLED)
else:
    print("  Nothing to do.")

# Step- Command the mount to AZ=90 deg and EL=80 deg.

* Command the mount to AZ=90 deg and EL=80 deg.
* Verify that the telemetry attached to this test step with their respective parameters are published and have reasonable values.

In [ ]:
await mount.cmd_moveToTarget.set_start(azimuth=90, elevation=80)

Step- Transition to Disabled State

In [ ]:
await salobj.set_summary_state(mount, salobj.State.DISABLED)
e = mount.evt_summaryState.get()
print("MTMount now in state:", e.summaryState) 

# Steps- Telemetry Verification

In [ ]:
e = vandv.check_last_evt(mount.tel_azimuth)

In [ ]:
e = await mount.tel_azimuth.next(flush=True, timeout=1)
print(e)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.azimuth", 
    fields="*",
    num=2,
)

print(df)

Verify the MTMount_azimuthDrives telemetry topic is being published to the EFD. Need to change the function. 
<br> **Note: This telemetry is not published yet.** 

In [ ]:
e = await mount.tel_azimuthDrives.next(flush=True, timeout=1)

In [ ]:
e = vandv.check_last_evt(mount.tel_azimuthDrives)

Verify that the MTMount_elevation telemetry topic is being published to the EFD.

In [ ]:
e = vandv.check_last_evt(mount.tel_elevation)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.elevation", 
    fields="*",
    num=2,
)

print(df)

Verify the MTMount_elevationDrives telemetry topic is being published to the EFD.
<br> **Note, this telemetry is not published yet.**

In [ ]:
e = await mount.tel_elevationDrives.next(flush=True, timeout=1)

Verify the MTMount_cameraCableWrap telemetry topic is being published to the EFD.

In [ ]:
e = vandv.check_last_evt(mount.tel_cameraCableWrap)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.cameraCableWrap", 
    fields="*",
    num=2,
)

print(df)

# Steps- Events Verification

Commandable By DDS Check
* Verify the MTMount is commandable by DDS by checking that the MTMount_logevent_commander event is published to the EFD with reasonably values.

In [ ]:
e = vandv.check_last_evt(mount.evt_commander)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_commander", 
    fields="commander",
    num=2,
)

print(df)

Verify the TCP/IP is connected to the low level controller. Note: **MTMount connected not published yet** 

In [ ]:
e = vandv.check_last_evt(mount.evt_connected)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_connected", 
    fields="*",
    num=5,
)

print(df)

Initial Warning Check
* Verify there are no warnings from any subsystem. **Can not be triggered now**. 
* TBD: Trigger a warning.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_warning", 
    fields="*",
    num=5,
)

print(df)

Verify the MTMount_logevent_availableSettings event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_availableSettings)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_availableSettings", 
    fields="*",
    num=2,
)

print(df)

Verify the MTMount_logevent_azimuthToppleBlock event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_azimuthToppleBlock)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_azimuthToppleBlock", 
    fields="*",
    num=2,
)

print(df)

Verify the MTMount_logevent_az0101CabinetThermalSystemState is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_az0101CabinetThermalSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_az0101CabinetThermalSystemState", 
    fields="*",
    num=2,
)

print(df)

Verify the MTMount_logevent_azimuthDrivesThermalSystemState is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_azimuthDrivesThermalSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_azimuthDrivesThermalSystemState", 
    fields="*",
    num=2,
)

print(df)

Verify the MTMount_logevent_elevationDrivesThermalSystemState is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_elevationDrivesThermalSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_elevationDrivesThermalSystemState", 
    fields="*",
    num=2,
)

print(df)

Verify the MTMount_logevent_topEndChillerSystemState is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_topEndChillerSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_topEndChillerSystemState", 
    fields="*",
    num=3,
)

print(df)

**System State Verification**
* Verify the MTMount_logevent_azimuthSystemState event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_azimuthSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_azimuthSystemState", 
    fields="*",
    num=2,
)

print(df)

Verify the MTMount_logevent_oilSupplySystemState event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_oilSupplySystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_oilSupplySystemState", 
    fields="*",
    num=3,
)

print(df)

Verify the MTMount_logevent_balanceSystemState event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_balanceSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_balanceSystemState", 
    fields="*",
    num=2,
)

print(df)

Verify the MTMount_logevent_cameraCableWrapSystemState event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_cameraCableWrapSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_cameraCableWrapSystemState", 
    fields="*",
    num=3,
)

print(df)

Verify the MTMount_logevent_deployablePlatformsSystemState event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_deployablePlatformsSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_deployablePlatformsSystemState", 
    fields="*",
    num=2,
)

print(df)

The MTMount_logevent_deployablePlatformsMotionState shows that the deployable platforms are not moving.

In [ ]:
e = vandv.check_last_evt(mount.evt_deployablePlatformsMotionState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_deployablePlatformsMotionState", 
    fields="*",
    num=2,
)

print(df)

Verify the MTMount_logevent_elevationSystemState event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_elevationSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_elevationSystemState", 
    fields="*",
    num=3,
)

print(df)

Verify that MTMount_logevent_mainAxesPowerSupplySystemState event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_mainAxesPowerSupplySystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_mainAxesPowerSupplySystemState", 
    fields="*",
    num=3,
)

print(df)

Verify that the MTMount_logevent_mainCabinetThermalSystemState event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_mainCabinetThermalSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_mainCabinetThermalSystemState", 
    fields="*",
    num=2,
)

print(df)

Verify that the MTMount_logevent_modbusTemperatureControllersSystemState event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_modbusTemperatureControllersSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_modbusTemperatureControllersSystemState", 
    fields="*",
    num=2,
)

print(df)

Verify that the MTMount_logevent_safetyInterlocks event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_safetyInterlocks)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_safetyInterlocks", 
    fields="*",
    num=2,
)

print(df)

**Limits Verification** 
* Verify the MTMount_logevent_elevationLimits event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_elevationLimits)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_elevationLimits", 
    fields="limits",
    num=2,
)

print(df)

Verify that the MTMount_logevent_azimuthLimits event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_azimuthLimits)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_azimuthLimits", 
    fields="limits",
    num=2,
)

print(df)

Verify that the MTMount_logevent_cameraCableWrapLimits event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_cameraCableWrapLimits)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_cameraCableWrapLimits", 
    fields="*",
    num=2,
)

print(df)

**Controller Settings Verification**
<br>Verify the MTMount_logevent_controllerSettingsName event is published to the EFD with reasonable parameters.
<br>**Note**: The following steps are verifying the events related to the settings file used in the low-level controller and do not have any direct commands that cause the events to be published. Therefore, these should be verified to be published to the EFD at startup.
<br> **Note: Not published yet**

In [ ]:
e = vandv.check_last_evt(mount.evt_controllerSettingsName)

Verify the MTMount_logevent_azimuthControllerSettings event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_azimuthControllerSettings)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_azimuthControllerSettings", 
    fields="*",
    num=2,
)

print(df)

Verify the MTMount_logevent_elevationControllerSettings event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_elevationControllerSettings)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_elevationControllerSettings", 
    fields="*",
    num=2,
)

print(df)

Verify the MTMount_logevent_cameraCableWrapControllerSettings event is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_cameraCableWrapControllerSettings)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_cameraCableWrapControllerSettings", 
    fields="*",
    num=2,
)

print(df)

# Mirror Cover Test

Verify that the Mirror Cover locks are not locked.

In [ ]:
e = vandv.check_last_evt(mount.evt_mirrorCoverLocksSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_mirrorCoverLocksSystemState", 
    fields="*",
    num=2,
)

print(df)

The MTMount_logevent_mirrorCoverLocksMotionState event is published to the EFD and shows the locks are not in motion either( parameters reversed in testcase?)

In [ ]:
e = vandv.check_last_evt(mount.evt_mirrorCoverLocksMotionState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_mirrorCoverLocksMotionState", 
    fields="*",
    num=5,
)

print(df)

While the mirror covers are open and the system is in Enabled mode, send an MTMount_command_closeMirrorCovers command through SAL.

In [ ]:
await salobj.set_summary_state(mount, salobj.State.ENABLED)
await mount.cmd_closeMirrorCovers.set_start()

While the mirror covers are in motion, the MTMount_logevent_mirrorCoversMotionState is published to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_mirrorCoversMotionState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_mirrorCoversMotionState", 
    fields="*",
    num=1,
)

print(df)

The MTMount_logevent_mirrorCoversSystemState is updated at the end of the command with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_mirrorCoversSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_mirrorCoversSystemState", 
    fields="*",
    num=1,
)

print(df)

With the mirror covers close, send an MTMount_command_openMirrorCovers command through SAL.

In [ ]:
await mount.cmd_openMirrorCovers.set_start()

In [ ]:
e = vandv.check_last_evt(mount.evt_mirrorCoversMotionState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_mirrorCoversMotionState", 
    fields="*",
    num=3,
)

print(df)

The MTMount_logevent_mirrorCoversSystemState is updated at the end of the command with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_mirrorCoversSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_mirrorCoversSystemState", 
    fields="*",
    num=2,
)

print(df)

**Camera Cable Wrap Test**
<br>Verify the MTMount_logevent_cameraCableWrapSystemState is in the Enabled state.

In [ ]:
e = vandv.check_last_evt(mount.evt_cameraCableWrapSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_cameraCableWrapSystemState", 
    fields="*",
    num=2,
)

print(df)

With the CCW still following the Rotator, send an MTMount_command_disableCameraCableWrapFollowing command through SAL.

In [ ]:
await mount.cmd_disableCameraCableWrapFollowing.set_start()

In [ ]:
e = vandv.check_last_evt(mount.evt_cameraCableWrapFollowing)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_cameraCableWrapFollowing", 
    fields="enabled",
    num=2,
)

print(df)

Send an MTMount_command_enablecameracableWrapFollowing command through SAL.

In [ ]:
await mount.cmd_enableCameraCableWrapFollowing.set_start()

In [ ]:
e = vandv.check_last_evt(mount.evt_cameraCableWrapFollowing)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_cameraCableWrapFollowing", 
    fields="enabled",
    num=2,
)

print(df)

# Move Test
<br>Before conducting any movement in elevation, verify the elevation Locking Pins are not engaged.

In [ ]:
e = vandv.check_last_evt(mount.evt_elevationLockingPinMotionState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_elevationLockingPinMotionState", 
    fields="*",
    num=2,
)

print(df)

The MTMount_logevent_lockingPinsSystemState event also indicates no locking pins are engaged for any of the motion controllers.

In [ ]:
e = vandv.check_last_evt(mount.evt_lockingPinsSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_lockingPinsSystemState", 
    fields="*",
    num=2,
)

print(df)

In the enabled state, send an MTMount_command_moveToTarget command of (30deg,30deg)

In [ ]:
await mount.cmd_moveToTarget.set_start(azimuth=30, elevation=30)

The command is accepted and the system publishes the MTMount_logevent_azimuthInPosition and MTMount_logevent_elevationInPosition events once the 
command is reached with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_azimuthInPosition)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_azimuthInPosition", 
    fields="inPosition",
    num=2,
)

print(df)

In [ ]:
e = vandv.check_last_evt(mount.evt_elevationInPosition)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_elevationInPosition", 
    fields="inPosition",
    num=2,
)

print(df)

The MTMount_logevent_azimuthMotionState and MTMount_logevent_elevationMotionState are published indicating they are in motion.

In [ ]:
e = vandv.check_last_evt(mount.evt_azimuthMotionState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_azimuthMotionState", 
    fields="state",
    num=2,
)

print(df)

In [ ]:
e = vandv.check_last_evt(mount.evt_elevationMotionState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_elevationMotionState", 
    fields="state",
    num=2,
)

print(df)

The MTMount_logevent_cameraCableWrapMotionState event is updated with reasonable parameters as the CCW follows during this motion

In [ ]:
e = vandv.check_last_evt(mount.evt_cameraCableWrapMotionState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_cameraCableWrapMotionState", 
    fields="state",
    num=2,
)

print(df)

Upon reaching the commanded position, the MTMount_logevent_cameraCableWrapInPosition event publishes to the EFD with reasonable parameters.

In [ ]:
e = vandv.check_last_evt(mount.evt_cameraCableWrapInPosition)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_cameraCableWrapInPosition", 
    fields="inPosition",
    num=2,
)

print(df)

The MTMount_logevent_azimuthCableWrapSystemState event publishes to the EFD with reasonable parameters as the Cable Wrap moves.

In [ ]:
e = vandv.check_last_evt(mount.evt_azimuthCableWrapSystemState)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_azimuthCableWrapSystemState", 
    fields="*",
    num=2,
)

print(df)

**Stop Test**
<br>Send a larger MTMount_command_moveToTarget command of (60deg,60deg)

In [ ]:
await mount.cmd_moveToTarget.set_start(azimuth=60, elevation=60)

While the system is still in motion, send a MTMount_command_stop command.

In [ ]:
await mount.cmd_stop.set_start()

# Tracking Test
<br>Send an MTMount_command_homeBothAxes command to home the elevation and azimuth axes.
<br>Notes: 
The encoder tapes have unique fiducial marks on the order of every few degrees.
Elevation homing only occurs in the positive direction so if this command is sent while the telescope is in zenith, then this will cause an issue.

In [ ]:
await mount.cmd_homeBothAxes.set_start()

The MTMount_azimuth and MTMount_elevation telemetry topics publish data to the EFD with reasonable values

In [ ]:
e = vandv.check_last_evt(mount.tel_azimuth)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.azimuth", 
    fields="*",
    num=2,
)

print(df)

In [ ]:
e = vandv.check_last_evt(mount.tel_elevation)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.elevation", 
    fields="*",
    num=2,
)

print(df)

Send an MTMount_command_startTracking command through SAL.

In [ ]:
await mount.cmd_startTracking.set_start()

# Track Target Command

Send an MTMount_command_trackTarget command. Need Example parameters. (Need to send more data at once using multi line in a single cell)

In [ ]:
await mount.cmd_trackTarget.set_start(azimuth=65, azimuthVelocity=0, elevation=65, elevationVelocity=0, taiTime=1.2, trackId =0 , tracksys = "local", radesys= "ICRS")

Need to modify the following script to create a function for sending multiple track targert commands. 

In [ ]:
STD_TIMEOUT = 2
TRACK_INTERVAL = 0.1 # interval between tracking updates (seconds)
# How far in advance to set the time field of tracking commands (seconds)
TRACK_ADVANCE_TIME = 0.05

In [ ]:
az = [30,35,40,45,50]
azVel = [0.2,0.4,0.6,0.8,1.0] 
el = [30,35,40,45,50]
elVel = [0.2,0.4,0.6,0.8,1.0] 
ti = [0.1,0.2,0.3,0.6,1.1]
tid = [1,2,3,4,5]
iteration = [0,1,2,3,4]
await mount.cmd_startTracking.set_start()
for i in iteration:
    await mount.cmd_trackTarget.set_start(
                    azimuth=az[i],
                    azimuthVelocity=azVel[i],
                    elevation=el[i],
                    elevationVelocity=elVel[i],
                    taiTime=ti[i],
                    trackId=tid[i],
                    tracksys="local",
                    radesys="ICRS",
                )
    await asyncio.sleep(TRACK_INTERVAL)


The MTMount_logevent_target event publishes the reached target to the EFD.

In [ ]:
e = vandv.check_last_evt(mount.evt_target)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_target", 
    fields="*",
    num=2,
)

print(df)

The CCW follows and publishes an MTMount_logevent_cameraCableWrapTarget event to the EFD with reasonable values.

In [ ]:
e = vandv.check_last_evt(mount.evt_cameraCableWrapTarget )

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_cameraCableWrapTarget", 
    fields="*",
    num=2,
)

print(df)

Send an MTAOS_command_stopTracking command through SAL.

In [ ]:
await mount.cmd_stopTracking.set_start()

# Error Test
<br>Send another MTMount_command_startTracking command.

In [ ]:
await mount.cmd_startTracking.set_start()

Wait until an error is published due to a lack of MTMount_command_trackTarget commands.

In [ ]:
e = vandv.check_last_evt(mount.evt_error)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTMount.logevent_error", 
    fields="*",
    num=2,
)

print(df)

In [ ]:
e = mount.evt_summaryState.get()
print("MTMount now in state:", e.summaryState)

Send an MTMount_command_clearError command to transition the system out of the FAULT state.

In [ ]:
await mount.cmd_clearError.set_start()

In [ ]:
print("  Transitioning to DISABLED state")
await salobj.set_summary_state(mount, salobj.State.DISABLED)

In [ ]:
e = mount.evt_summaryState.get()
print("MTMount now in state:", e.summaryState)